In [ ]:
import pandas as pd
import numpy as np
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.metrics import get_time_dependent_gt

mall = []
label_data = pd.read_csv(get_param_in_cwd('survival_file'), dtype={'ID': str})
label_data = pd.merge(label_data, pd.read_csv('joinit_group.csv')[['ID']])
label_data

In [ ]:
import matplotlib.pyplot as plt
from onekey_algo.custom.components.comp1 import merge_results, draw_roc
from onekey_algo.custom.components.metrics import analysis_pred_binary
from onekey_algo.custom.components.comp1 import draw_calibration
from onekey_algo.custom.components.comp1 import plot_DCA
metric = []
youden = {}

sname = 'PFS'
model_names = ['Full'+sname]

def get_prediction(mn, subset):
    prediction = pd.read_csv(f'results/{mn}_cox_predictions_{subset}.csv')
    prediction.columns = ['ID', f'{mn}_Exp', mn]
    return prediction[['ID', f'{mn}']]

for time, tname in [(1*12, '1 Year'), (3*12, '3 Years'), (5*12, '5 Years')]:
    gts = []
    preds = []
    for subset in get_param_in_cwd('subsets'):
        data = [get_prediction(mn, subset)for mn in model_names]
        time_label_data = get_time_dependent_gt(label_data, time=time, 
                                                duration_col=f"{sname}_time", event_col=sname)
        ALL_results = merge_results(*(data + [time_label_data]))
#         display(ALL_results)
        ALL_results = ALL_results.drop_duplicates('ID')

        gt = [1-np.array(ALL_results['label']) for _ in model_names]
        pred_train = [np.array(ALL_results[d]) for d in model_names]
        gts.extend(gt)
        preds.extend(pred_train)
        draw_roc(gt, pred_train, labels=model_names, title=f'Cohort {subset} {tname} ROC')
        plt.savefig(f'img/{tname}_{sname}_{subset}_auc.svg')
        plt.show()
        for mname, y, score in zip(model_names, gt, pred_train):
            # 计算验证集指标
            acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
            ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
            metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, tname, f"{subset[0].upper() + subset[1:]}"))
            youden[mname] = thres
    draw_roc(gts, preds, labels=get_param_in_cwd('subsets'), title=f"{tname} Time Dependent ROC")
    plt.savefig(f'img/{tname}_{sname}_auc.svg')
    plt.show()
#         draw_calibration(pred_scores=pred_train, n_bins=5, remap=True,
#                          y_test=gt, model_names=model_names)
#         plt.title(f"Cohort {subset} {tname} Calibration")
#         plt.savefig(f'img/{tname.split(" ")}_{subset}_cali.svg')
#         plt.show()
        
#         plot_DCA([ALL_results[model_name] for model_name in model_names], 
#                  ALL_results['label'], title=f'Cohort {subset} {tname} DCA', labels=model_names, y_min=-0.15)
#         plt.savefig(f'img/{tname.split(" ")}_{subset}_dca.svg')
#         plt.show()
mall.append(pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                                      'PPV', 'NPV', 'Survival', 'Cohort']))

In [ ]:
import matplotlib.pyplot as plt
from onekey_algo.custom.components.comp1 import merge_results, draw_roc
from onekey_algo.custom.components.metrics import analysis_pred_binary
from onekey_algo.custom.components.comp1 import draw_calibration
from onekey_algo.custom.components.comp1 import plot_DCA
metric = []
youden = {}

sname = 'OS'
model_names = ['Full'+sname]

def get_prediction(mn, subset):
    prediction = pd.read_csv(f'results/{mn}_cox_predictions_{subset}.csv')
    prediction.columns = ['ID', f'{mn}_Exp', mn]
    return prediction[['ID', f'{mn}']]

for time, tname in [(3*12, '3 Years'), (5*12, '5 Years')]:
    gts = []
    preds = []
    for subset in get_param_in_cwd('subsets'):
        data = [get_prediction(mn, subset)for mn in model_names]
        time_label_data = get_time_dependent_gt(label_data, time=time, 
                                                duration_col=f"{sname}_time", event_col=sname)
        ALL_results = merge_results(*(data + [time_label_data]))
        ALL_results = ALL_results.drop_duplicates('ID')

        gt = [1-np.array(ALL_results['label']) for _ in model_names]
        pred_train = [np.array(ALL_results[d]) for d in model_names]
        gts.extend(gt)
        preds.extend(pred_train)
        draw_roc(gt, pred_train, labels=model_names, title=f'Cohort {subset} {tname} ROC')
        plt.savefig(f'img/{tname}_{sname}_{subset}_auc.svg')
        plt.show()
        for mname, y, score in zip(model_names, gt, pred_train):
            # 计算验证集指标
            acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y, score)
            ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
            metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, tname, f"{subset[0].upper() + subset[1:]}"))
            youden[mname] = thres
    draw_roc(gts, preds, labels=get_param_in_cwd('subsets'), title=f"{tname} Time Dependent ROC")
    plt.savefig(f'img/{tname}_{sname}_auc.svg')
    plt.show()
#         draw_calibration(pred_scores=pred_train, n_bins=5, remap=True,
#                          y_test=gt, model_names=model_names)
#         plt.title(f"Cohort {subset} {tname} Calibration")
#         plt.savefig(f'img/{tname.split(" ")}_{subset}_cali.svg')
#         plt.show()
        
#         plot_DCA([ALL_results[model_name] for model_name in model_names], 
#                  ALL_results['label'], title=f'Cohort {subset} {tname} DCA', labels=model_names, y_min=-0.15)
#         plt.savefig(f'img/{tname.split(" ")}_{subset}_dca.svg')
#         plt.show()
mall.append(pd.DataFrame(metric, index=None, columns=['Signature', 'Accuracy', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 
                                                      'PPV', 'NPV', 'Survival', 'Cohort']))

In [ ]:
pd.concat(mall, axis=0)